# Datengenerierung

Dieses Notebook stellt die Code bereit, um aus den im Ordner `ntv-data` liegenden NTV-Nachrichtenartikeln eine `data.csv`-Datei zu generieren, die anschließend im Notebook `datenanalyse.ipynb` als Datengrundlage für weitere Analysen verwendet werden kann. Das Einlesen der 5000+ Nachrichtenartikel führt aufgrund der vielen Dateizugriffe zu einer schlechten Performance. Es ist daher sinnvoller, sobald neue Daten hinzukommen, eine einzelne `.csv`-Datei zu generieren, die anschließend mit einem einzelnen Dateizugriff mit bspw. `pandas` eingelesen werden kann.

## Benötige Module

Zunächst müssen die benötigten Python-Packages importiert werden. Zu installieren ist hierfür zunächst das Package `BeautifulSoup` mit folgendem `pip`-Aufruf:

```pip install beautifulsoup4```

Anschließend können die Packages importiert werden.

In [30]:
import os
import re
import csv
from datetime import datetime
from typing import Dict, List
from bs4 import BeautifulSoup

## Hilfsfunktionen

Die Funktion `get_content_of` nimmt einen Dateipfad entgegen und gibt den entsprechenden Inhalt als `string` zurück.

In [31]:
def get_content_of(article_path: str) -> str:
    result = ""
    with open(article_path, "r", encoding="utf-8") as article:
        result = article.read()
    if result != "":
        return result
    raise Exception("File not found")

Mit Hilfe der `extract_text_from_html_article`-Funktion extrahieren wir den eigentlichen Nachrichtentext aus der `html`-Datei und geben diesen als `string` zurück.

In [32]:
def extract_text_from_html_article(html_text: str) -> str:
    soup = BeautifulSoup(html_text, "html.parser")
    article_text = soup.find("div", class_="article__text")
    if article_text:
        soup = BeautifulSoup(article_text.text, "html.parser")
        for aside in soup.find_all("div", class_="article__aside"):
            aside.decompose()
        output = soup.text.strip()
        output = re.sub(" +", " ", output)
        output = output.replace("\n", "")
        return output
    raise Exception("No article text was found")

In [33]:
def extract_exact_date_from_html_article(html_text: str) -> str:
    soup = BeautifulSoup(html_text, "html.parser")
    date = soup.find("span", class_="article__date")
    if date:
        output = date.get_text()
        return output
    print("No date was found")
    return None

Die Überschrift des jeweiligen Artikels kann aus dem Dateinamen abgeleitet werden. Die Funktion `convert_filename_to_article_headline` erfüllt genau diese Funktion.

In [34]:
def convert_filename_to_article_headline(article_filename: str) -> str:
    headline = (" ").join(article_filename.split("-")[:-1])
    return headline

In [35]:
def parse_ntv_datetime(date_string: str):
    try:
        datetime_obj = datetime.strptime(date_string, "%d.%m.%Y, %H:%M Uhr")
        date = datetime_obj.date()
        time = datetime_obj.time()
        return date, time
    except:
        return None, None


Die letzte Funktion, `collect_article_contents_from`, sammelt nun, unter Verwendung der obigen Funktionen, die Aritkel und generiert eine Liste, in der die jeweiligen Einträge jeweils die Informationen und Inhalte eines Artikels darstellen.

In [36]:
def collect_article_contents_from(path: str) -> List[Dict[str, str]]:
    collection = []
    dates = os.listdir(path)
    count_articles = 0
    print()
    for date in dates:
        categories = os.listdir(f"{path}/{date}")
        for category in categories:
            article_list = os.listdir(f"{path}/{date}/{category}")
            for article in article_list:
                if article != "rss.json" and article != ".ipynb_checkpoints":
                    count_articles += 1
                    print('\r', end='')
                    print(f"Scanned articles: {count_articles} - scanning: {article}", end='')
                    article_content = get_content_of(f"{path}/{date}/{category}/{article}")
                    article_text = extract_text_from_html_article(article_content)
                    article_extracted_date = extract_exact_date_from_html_article(article_content)
                    article_date, article_time = parse_ntv_datetime(article_extracted_date)
                    collection.append(
                        {
                            "date": article_date,
                            "time": article_time,
                            "category": category,
                            "headline": convert_filename_to_article_headline(article),
                            "filename": article,
                            "text": article_text,
                        }
                    )
    return collection

## Datengenerierung

Damit kann nun der Inhalt der Artikel gesammelt werden und anschließend mit Hilfe des Packages `csv` die `1-1-data.csv`-Datei mit Nachrichtenartikeln generiert werden.

In [37]:
content = collect_article_contents_from("ntv-data")
with open("2-1-data.csv", "w", encoding="utf-8", newline="") as output_file:
    fc = csv.DictWriter(output_file, fieldnames=content[0].keys())
    fc.writeheader()
    fc.writerows(content)


Scanned articles: 1022 - scanning: Ulm-im-Pokal-gegen-die-Bayern-VfB-in-Muenster-gefordert-article24982731.htmlhtmllhtmlrticle24976528.html8.htmlml4993852.htmltml1.htmlhtmlticle24999004.html

/var/folders/1m/1kc7bh856v32jwgwrwc239g00000gn/T/ipykernel_62852/2097671092.py:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(article_text.text, "html.parser")


Scanned articles: 10683 - scanning: Zoelle-gegen-China-Autos-wuerden-uns-aermer-machen-article25003580.htmlmlhtmlhtmll-Schlaganfall-und-Herzinfarkt-article25005778.htmln-Frankenstein-Einheiten-helfen-article25015701.html